In [29]:
import os
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from nltk.tokenize import word_tokenize
from string import punctuation

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, auc, roc_auc_score
from sklearn.externals import joblib

import scipy
from scipy.sparse import hstack

In [17]:
df = pd.read_csv('C:/Users/vbigand/Downloads/twitter_sentiment.csv', encoding='latin1', usecols=['Sentiment','SentimentText'])
df.columns = ['sentiment','text']
df = df.sample(frac=1,random_state=1)
print(df.shape)

(1578614, 2)


In [18]:
for i in range(10):
    print(df.iloc[i]['sentiment'],df.iloc[i]['text'])

1 @OtaliaRocks You mean looky 
0 @lecruetlecuit I saw that 
1 @SoulIsTheGoal not yet doh! watching Big Bang Theory S1 dan lent me \o/ 
0 @xbllygbsn lol saaame :$. wish they were healthy  lol :$. aah okayy :$. dont drink bottled water from asda then ;)
1 I'm pleased with myself. I've only just got up and it's 10:03. Score! 
1 @elissaalva HEEEEYYYYY GIIIRRRRRRLLLLL!!!!!!! I hope all is well with you 
0 Napped and missed the gym! Busy day, bed early! Class tomorrow 
0 @feybee Haha g2g to sleep now  my cus said so ttyl love ya!! Good Noght God Bless!!
1 I am contimplating throwing myself a birthday part @ a rollerskating rink to force all my friends to rollerskate with me  who would come?
1 #jonasnewsongs favorites: fly with me and much better. you boys rock  marry me nick?


In [19]:
#other method
for row in df.head(10).iterrows():
    print(row[1].sentiment,row[1].text)

1 @OtaliaRocks You mean looky 
0 @lecruetlecuit I saw that 
1 @SoulIsTheGoal not yet doh! watching Big Bang Theory S1 dan lent me \o/ 
0 @xbllygbsn lol saaame :$. wish they were healthy  lol :$. aah okayy :$. dont drink bottled water from asda then ;)
1 I'm pleased with myself. I've only just got up and it's 10:03. Score! 
1 @elissaalva HEEEEYYYYY GIIIRRRRRRLLLLL!!!!!!! I hope all is well with you 
0 Napped and missed the gym! Busy day, bed early! Class tomorrow 
0 @feybee Haha g2g to sleep now  my cus said so ttyl love ya!! Good Noght God Bless!!
1 I am contimplating throwing myself a birthday part @ a rollerskating rink to force all my friends to rollerskate with me  who would come?
1 #jonasnewsongs favorites: fly with me and much better. you boys rock  marry me nick?


In [20]:
#re.sub let us replace the pattern by the string of your choice
def tokenize(tweet):
    tweet = re.sub(r'http\S+','',tweet)
    tweet = re.sub(r'@(\w+)','',tweet)
    tweet = re.sub(r'#(\w+)','',tweet)
    tweet = re.sub(r'[^\w\s]', '', tweet)
    tweet = tweet.strip().lower()
    tokens = word_tokenize(tweet)
    return tokens   

In [21]:
df['tokens'] = df.text.map(tokenize)

In [22]:
df['cleaned_text'] = df['tokens'].map(lambda tokens: ' '.join(tokens))

In [23]:
df[['sentiment','cleaned_text']].to_csv('C:/Users/vbigand/Downloads/clean_twitter_text')

df = pd.read_csv('C:/Users/vbigand/Downloads/clean_twitter_text')
df.drop('Unnamed: 0',axis=1,inplace=True)
print(df.shape)
df.head()

(1578614, 2)


,sentiment,cleaned_text
0,1,you mean looky
1,0,i saw that
2,1,not yet doh watching big bang theory s1 dan le...
3,0,lol saaame wish they were healthy lol aah okay...
4,1,im pleased with myself ive only just got up an...


## Word tokens

In [24]:
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_text'].values.astype('str'),df['sentiment'].values.astype('str'),random_state=42, test_size = 0.1, stratify = df['sentiment'])
print ("X_train: {}\ty_train: {}\nX_test: {}\ty_test: {}".format(X_train.shape,y_train.shape,X_test.shape,y_test.shape))

X_train: (1420752,)	y_train: (1420752,)
X_test: (157862,)	y_test: (157862,)


In [33]:
vect = TfidfVectorizer(max_features=40000,min_df=5,max_df=0.8,stop_words='english',analyzer='word',ngram_range=(1,2))
vect.fit(X_train)
tfidf_mat_x_train = vect.transform(X_train)
tfidf_mat_x_test = vect.transform(X_test)

In [1]:
clf = LogisticRegression(solver='sag',verbose=2)
clf.fit(tfidf_mat_x_train,y_train)

NameError: name 'LogisticRegression' is not defined

In [35]:
y_predict = clf.predict(tfidf_mat_x_test)
print('Accuracy score: {}'.format(accuracy_score(y_pred=y_predict,y_true=y_test)))

Accuracy score: 0.781480026858902


## Char tokens

In [36]:
vect = TfidfVectorizer(max_features=40000,min_df=5,max_df=0.5,stop_words='english',analyzer='char',ngram_range=(1,4))
vect.fit(X_train)
tfidf_mat_x_train_char = vect.transform(X_train)
tfidf_mat_x_test_char = vect.transform(X_test)
clf = LogisticRegression(solver='sag',verbose=2)
clf.fit(tfidf_mat_x_train,y_train)
y_predict = clf.predict(tfidf_mat_x_test)
print('Accuracy score: {}'.format(accuracy_score(y_pred=y_predict,y_true=y_test)))

convergence after 21 epochs took 78 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.3min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.3min finished


Accuracy score: 0.8047471842495344


## Word + Char tokens

In [41]:
tfidf_word_char_x_train = hstack((tfidf_mat_x_train_char,tfidf_mat_x_train))
tfidf_word_char_x_test = hstack((tfidf_mat_x_test_char,tfidf_mat_x_test))
clf = LogisticRegression(solver='sag',verbose=2)
clf.fit(tfidf_word_char_x_train, y_train)
y_predict = clf.predict(tfidf_word_char_x_test)
print('Accuracy score: {}'.format(accuracy_score(y_pred=y_predict,y_true=y_test)))

MemoryError: 